In [20]:
library(bigrquery)
library(tidyverse)

In [21]:
# Summary Statistics: Ethnicity
projectid = "yhcr-prd-phm-bia-core"
sql <- "SELECT distinct ethnicity_source_value FROM `yhcr-prd-phm-bia-core.CB_FDM_PrimaryCare_v5.person`"
tb <- bq_project_query(projectid, sql)
eth <-bq_table_download(tb)

In [22]:
eth

ethnicity_source_value
<chr>
NULL
Unknown/Refuse to say
White:Any other White background
Asian or Asian British: Bangladeshi - England and Wales ethnic category 2011 census
Asian or Asian British: Chinese - England and Wales ethnic category 2011 census
Asian or Asian British: Indian - England and Wales ethnic category 2011 census
Asian or Asian British: Pakistani - England and Wales ethnic category 2011 census
Asian or Asian British: any other Asian background - England and Wales ethnic category 2011 census
Black or African or Caribbean or Black British: African - England and Wales ethnic category 2011 census


In [23]:
# Map to 5 category ethnicity 2011 census
eth <- eth %>%
    mutate(eth_5cat = case_when(
    ethnicity_source_value == "NULL" | ethnicity_source_value == "Unknown/Refuse to say" ~ NA,
    ethnicity_source_value == "Asian or Asian British: Bangladeshi - England and Wales ethnic category 2011 census" | ethnicity_source_value == "Asian or Asian British: Chinese - England and Wales ethnic category 2011 census" | ethnicity_source_value == "Asian or Asian British: Indian - England and Wales ethnic category 2011 census" | ethnicity_source_value == "Asian or Asian British: Pakistani - England and Wales ethnic category 2011 census" | ethnicity_source_value == "Asian or Asian British: any other Asian background - England and Wales ethnic category 2011 census" ~ 1, # Asian
    ethnicity_source_value == "Black or African or Caribbean or Black British: African - England and Wales ethnic category 2011 census" | ethnicity_source_value == "Black or African or Caribbean or Black British: Caribbean - England and Wales ethnic category 2011 census" | ethnicity_source_value == "Black or African or Caribbean or Black British: other Black or African or Caribbean background - England and Wales ethnic category 2011 census" ~ 2, # Black
    ethnicity_source_value == "Mixed multiple ethnic groups: White and Asian - England and Wales ethnic category 2011 census" | ethnicity_source_value == "Mixed multiple ethnic groups: White and Black African - England and Wales ethnic category 2011 census" | ethnicity_source_value == "Mixed multiple ethnic groups: White and Black Caribbean - England and Wales ethnic category 2011 census" | ethnicity_source_value == "Mixed multiple ethnic groups: any other Mixed or multiple ethnic background - England and Wales ethnic category 2011 census" ~ 3, # Mixed
    ethnicity_source_value == "White:Any other White background" | ethnicity_source_value == "White: English or Welsh or Scottish or Northern Irish or British - England and Wales ethnic category 2011 census" | ethnicity_source_value == "White: Gypsy or Irish Traveller - England and Wales ethnic category 2011 census" | ethnicity_source_value == "White: Irish - England and Wales ethnic category 2011 census" ~ 4, # White
    ethnicity_source_value == "Other ethnic group: Arab - England and Wales ethnic category 2011 census" | ethnicity_source_value == "Other ethnic group: any other ethnic group - England and Wales ethnic category 2011 census" ~ 5)) # Other

In [16]:
bq_auth()

In [17]:
colnames(eth)

[1] "ethnicity_source_value" "eth_5cat"

In [18]:
# set destination table - do this regardless of whether it already exists
dest_dataset <- bq_dataset("yhcr-prd-phm-bia-core.CB_LOOKUPS")
dest_table <- bq_table(dest_dataset, "tbl_ethnicity_2011_census")

# create a full table spec as follows
# then pass this to bq_table_upload
dest_fields <- bq_fields(list(bq_field("ethnicity_source_value", "STRING", "REQUIRED"),
                              bq_field("eth_5cat", "INT64")))

# delete an existing table if necessary
if(bq_table_exists(dest_table)) bq_table_delete(dest_table)
# upload file_data to dest_table
bq_table_upload(dest_table, eth, fields = dest_fields, create_disposition='CREATE_IF_NEEDED', write_disposition='WRITE_TRUNCATE')

ERROR: Error in eval(assertion, env): argument "dataset" is missing, with no default


In [19]:
write_csv(eth, file = "tbl_Ethnicity_2011_census.csv")